# Feature Engineering

## 1.0 Imports

In [1]:
# High-performance library for numerical computing
# Supports multi-dimensional arrays, linear algebra, and high-performance math operations
import numpy as np

# Versatile data analysis toolkit built on NumPy
# Provides powerful tools for handling, merging and analyzing structured data
import pandas as pd

## 2.0 Multi Table Data Preparation

### 2.1 Account Table
Static characteristics of accounts.

| Item | Meaning | Remark |
|:------|:------|:------|
| `account_id` | Unique account identifier ||
| `district_id` | Branch location ||
| `frequency` | Statement issuance frequency | "POPLATEK MESICNE" = Monthly <br> "POPLATEK TYDNE" = Weekly <br> "POPLATEK PO OBRATU" = After transaction |
| `date` | Account creation date | Format: YYMMDD |

In [2]:
# Reads raw account data from a semicolon-separated file
account = pd.read_csv(
    '../data/raw-files/account.asc',
    sep=';',
    # Automatically parses date strings into basic datetime objects for easier time-based operations
    parse_dates=['date'],
    date_format='%y%m%d'
    # Renames columns for better clarity and consistency
).rename(columns={'district_id': 'branch_district', 'date': 'account_date'})

In [3]:
# Translation dictionary for Czech terms – continuously expanding throughout the notebook
dictionary = {
    'POPLATEK MESICNE': 'monthly issuance',
    'POPLATEK TYDNE': 'weekly issuance',
    'POPLATEK PO OBRATU': 'issuance after transaction'
}
account['frequency'] = account['frequency'].map(dictionary)

### 2.2 Loan Table
Details of loans granted to each account.

| Item | Meaning | Remark |
|:------|:------|:------|
| `loan_id` | Unique loan identifier ||
| `account_id` | Unique account identifier ||
| `date` | Loan grant date | in the form YYMMDD |
| `amount` | Loan amount ||
| `duration` | Loan duration ||
| `payments` | Monthly loan payments ||
| `status` | Loan repayment status | 'A' = Contract finished, no issues <br> 'B' = Contract finished, loan unpaid <br> 'C' = Active contract, all good <br> 'D' = Active contract, client in debt |

In [4]:
loan = pd.read_csv(
    '../data/raw-files/loan.asc',
    sep=';',
    usecols=['account_id', 'date', 'amount', 'duration', 'status'],
    parse_dates=['date'],
    date_format='%y%m%d'
).rename(columns={'date': 'loan_date'})

For now, we're only focusing on finished contracts, since we're interested in a classic credit default vs. non-default scenario. While ongoing contracts could technically fit into this approach, their current status wouldn't give us a reliable read on the future.

In [5]:
loan = loan.query('status in ["A", "B"]')

### 2.3 Transaction Table

Dynamic characteristics of transactions in accounts.

| Item | Meaning | Remark |
|:------|:------|:------|
| `trans_id` | Unique transaction identifier ||
| `account_id` | Unique account identifier | The account involved in the transaction |
| `date` | Transaction date | Format: YYMMDD |
| `type` | Transaction type (+/-) | "PRIJEM" = Credit <br> "VYDAJ" = Withdrawal |
| `operation` | Transaction method | "VYBER KARTOU" = Credit card withdrawal  <br> "VKLAD" = Cash deposit <br> "PREVOD Z UCTU" = Incoming bank transfer <br> "VYBER" = Cash withdrawal <br> "PREVOD NA UCET" = Outgoing bank transfer |
| `amount` | Transaction amount ||
| `balance` | Account balance after transaction ||
| `k_symbol` | Transaction category | "POJISTNE" = Insurance payment <br> "SLUZBY" = Payment for statement <br> "UROK" = Interest credited <br> "SANKC. UROK" = Sanction interest (negative balance) <br> "SIPO" = Household expenses <br> "DUCHOD" = Pension payment <br> "UVER" = Loan repayment |
| `bank` | Partner’s bank | Each bank has a unique two-letter code |
| `account` | Partner’s account ||

In [6]:
transaction = pd.read_csv(
    '../data/raw-files/trans.asc',
    sep=';',
    usecols=['account_id', 'date', 'type', 'amount', 'balance', 'k_symbol'],
    parse_dates=['date'],
    date_format='%y%m%d'
).rename(columns={'date': 'trans_date'})

The table initially includes transactions from all accounts, regardless of loan approval. To ensure meaningful analysis, we focus only on transactions up to the month before loan approval, excluding the loan month itself. This prevents distortions from incomplete transaction patterns.

In [7]:
# Performs an INNER JOIN to retain only account_ids with granted loans present in both tables
transaction = loan[['account_id', 'loan_date']].merge(transaction, on='account_id', how='inner')

# Converts loan and transaction dates into monthly periods
transaction = transaction.assign(
    loan_month=transaction['loan_date'].dt.to_period('M'),
    trans_month=transaction['trans_date'].dt.to_period('M')
)

transaction = transaction.query('trans_month < loan_month')

To further ensure consistency, we also exclude each account's opening month, avoiding biases from incomplete early activity.

In [8]:
# Identifies the first transaction month for each account
opening_months = transaction.groupby('account_id').agg(opening_month=('trans_month', 'first'))
transaction = transaction.merge(opening_months, on='account_id')

transaction = transaction.query('opening_month < trans_month')

In [9]:
# Returns unique transaction types
transaction['type'].unique()

array(['PRIJEM', 'VYDAJ', 'VYBER'], dtype=object)

The unique transaction types appear to be somewhat mixed up, with extra entries beyond the original definitions. However, since _VYBER_ refers to cash withdrawals, we can confidently categorize it as withdrawal in this context.

In [10]:
dictionary.update({
    'PRIJEM': 'deposit',
    'VYDAJ': 'withdrawal',
    'VYBER': 'withdrawal'
})
transaction['type'] = transaction['type'].map(dictionary)

We'll also break down deposits and withdrawals separately to get clearer insights — since they each come with their own patterns and risks.

In [11]:
transaction['k_symbol'].unique()

array([nan, 'UROK', 'SLUZBY', 'POJISTNE', ' ', 'SIPO', 'SANKC. UROK'],
      dtype=object)

In [12]:
dictionary.update({
    'UROK': 'interest credited',
    'SIPO': 'household',
    'POJISTNE': 'insurance',
    'SANKC. UROK': 'sanction interest',
    ' ': np.nan
})
transaction['k_symbol'] = transaction['k_symbol'].map(dictionary)

#### 2.3.1 Deposit Sub Table

In [13]:
# Filters all transactions that are classified as deposit, focusing on relevant columns
deposit = transaction.query('type == "deposit"')[['account_id', 'trans_month', 'amount', 'k_symbol']]

In [14]:
deposit['k_symbol'].unique()

array([nan, 'interest credited'], dtype=object)

When it comes to unclassified deposits, we classify them as _income_ by default. In theory, interest earnings could fall under the same umbrella — but I find it more interesting to keep them separate.

Interest is a passive reward — your old money continuing to work for you — while _income_ introduces new money, actively increasing the balance. And while both contribute to financial growth, distinguishing between them could help to better understand the dynamics of money inflows.

In [15]:
deposit['k_symbol'] = deposit['k_symbol'].fillna('income')

In [16]:
# Splits deposit amounts across the categories in k_symbol
deposit = deposit.pivot_table(
    # Groups data by account and transaction month
    index=['account_id', 'trans_month'],
    # Converts the categories in k_symbol into separate columns
    columns='k_symbol',
    # Defines the transaction amount to be aggregated
    values='amount',
    # Aggregates the amounts by summing them for each category
    aggfunc='sum',
    # Fills in missing values when a category has no transactions for the month
    fill_value=0
)
deposit

k_symbol                 income  interest credited
account_id trans_month                            
2          1993-03      23936.0               13.5
           1993-04      20236.0              109.5
           1993-05      20236.0              144.7
           1993-06      30354.0              159.9
           1993-07      20236.0              203.0
...                         ...                ...
11362      1996-07      16141.0               93.2
           1996-08      16141.0               75.2
           1996-09      16141.0               73.4
           1996-10      16141.0               86.8
           1996-11      16141.0               94.3

[2584 rows x 2 columns]

In [17]:
# Aggregates deposit data for each account over all months
deposit = (
    deposit.groupby('account_id')
    # Calculates key statistical metrics for each account
    .agg(
        credits=('interest credited', 'mean'),
        income=('income', 'mean'),
        std_income=('income', 'std'),
        # Pulls the most recent income data per account
        last_income=('income', 'last')
    )
    # Restores account_id as a column after aggregation by resetting the index
    .reset_index()
    # Fills in missing values that may result from statistical calculations
    # For example, NaN can appear in the standard deviation if there is only a single value
    .fillna(0)
)
deposit

,account_id,credits,income,std_income,last_income
0,2,128.930000,22629.600000,4231.425271,30354.0
1,19,249.090909,18678.818182,3407.710106,17739.0
2,25,255.268750,62760.562500,44156.718371,36216.0
3,67,193.711111,44947.500000,7955.344347,41490.0
4,97,157.021429,20356.000000,4525.256576,17928.0
...,...,...,...,...,...
229,11141,184.000000,42507.000000,25704.345988,52556.0
230,11231,286.171429,47831.714286,8436.545002,44643.0
231,11265,90.257143,11319.285714,3989.746638,12918.0
232,11359,168.052381,29586.761905,5434.864376,27014.0


#### 2.3.2 Withdrawal Sub Table

In [18]:
# Filters all transactions that are classified as withdrawal, focusing on relevant columns
withdrawal = transaction.query('type == "withdrawal"')[['account_id', 'trans_month', 'amount', 'k_symbol']]

In [19]:
withdrawal['k_symbol'].unique()

array([nan, 'insurance', 'household', 'sanction interest'], dtype=object)

I'd rather not set up another pivot table for this since we can simply categorize everything as expenses. Household and insurance data don't seem particularly insightful here. The only additional detail we need is whether anyone has ever paid sanctioned interest due to a negative balance.

In [20]:
# Aggregates withdrawals of all accounts for each month
withdrawal = (
    withdrawal.groupby(['account_id', 'trans_month'])
    .agg(
        expenses=('amount', 'sum'),
        # Flags accounts with any recorded sanction interest, returning a boolean indicator
        sanctions=('k_symbol', lambda x: any(x == 'sanction interest'))
    )
)
withdrawal

expenses  sanctions
account_id trans_month                     
2          1993-04       11000.0      False
           1993-05       17600.0      False
           1993-06       22400.0      False
           1993-07       26659.6      False
           1993-08       32466.6      False
...                          ...        ...
11362      1996-07       18789.6      False
           1996-08       18049.6      False
           1996-09       15109.6      False
           1996-10       12889.6      False
           1996-11       13509.6      False

[2423 rows x 2 columns]

In [21]:
# Aggregates withdrawal data for each account over all months
withdrawal = (
    withdrawal.groupby('account_id')
    .agg(
        expenses=('expenses', 'mean'),
        std_expenses=('expenses', 'std'),
        last_expenses=('expenses', 'last'),
        # Flags accounts again with any recorded sanction interest
        sanctions=('sanctions', 'any')
    )
    .reset_index()
    .fillna(0)
)
withdrawal

,account_id,expenses,std_expenses,last_expenses,sanctions
0,2,22316.066667,8143.521513,34476.6,False
1,19,20812.480000,15464.987177,21314.6,False
2,25,64609.560000,41476.355637,88105.6,False
3,67,43864.022222,17386.321724,40555.6,False
4,97,20424.738462,8650.948604,13979.6,False
...,...,...,...,...,...
227,11141,36358.500000,50389.806055,24784.0,False
228,11231,48500.200000,27212.235431,69035.6,False
229,11265,10423.800000,8906.282262,12947.6,False
230,11359,29702.371429,8132.716730,32749.6,False


#### 2.3.3 Balance Sub Table

In [22]:
# Aggregates balance of all accounts for each month
balance = (
    transaction.groupby(['account_id', 'trans_month'])
    .agg(
        balance=('balance', 'mean'),
        std_balance=('balance', 'std'),
        min_balance=('balance', 'min'),
        max_balance=('balance', 'max'),
        last_balance=('balance', 'last')
    )
    .reset_index()
)
balance

,account_id,trans_month,balance,std_balance,min_balance,max_balance,last_balance
0,2,1993-03,23807.166667,2140.103755,21336.0,25049.5,25049.5
1,2,1993-04,37988.633333,6319.508640,34285.5,45285.5,34394.9
2,2,1993-05,42945.800000,10119.852075,37030.9,54630.9,37175.6
3,2,1993-06,52649.566667,12886.734885,45129.6,67529.6,45289.5
4,2,1993-07,45104.733333,9313.280046,38880.5,62325.5,39083.5
...,...,...,...,...,...,...,...
2579,11362,1996-07,23044.236364,5795.899136,16707.2,35403.6,17921.8
2580,11362,1996-08,22097.900000,7303.857479,14067.2,30208.2,14888.5
2581,11362,1996-09,23469.344444,6582.588670,15919.9,31014.9,15993.2
2582,11362,1996-10,26197.800000,5968.873872,19244.6,32119.6,19331.4


In [23]:
# Aggregates monthly account balances into key reference metrics for all accounts
balance = (
    balance.groupby('account_id')
    .agg(
        balance=('balance', 'mean'),
        std_balance=('std_balance', 'mean'),
        min_balance=('min_balance', 'min'),
        max_balance=('max_balance', 'max'),
        last_balance=('last_balance', 'last')
    )
    .reset_index()
)
balance

,account_id,balance,std_balance,min_balance,max_balance,last_balance
0,2,34712.963175,8182.558442,10508.0,67529.6,27855.2
1,19,29237.219300,8439.779831,3592.7,58157.5,16315.0
2,25,60429.613832,17896.754239,16700.0,119652.7,40024.5
3,67,55077.743519,17781.971353,17413.7,107069.6,23703.8
4,97,36650.692720,8086.971763,11502.9,60334.7,40548.9
...,...,...,...,...,...,...
229,11141,50877.992381,11500.270510,13467.0,106467.0,80916.0
230,11231,66444.853900,20364.737397,29988.5,109529.5,43003.3
231,11265,23911.735714,3688.610718,8618.0,41469.1,18338.6
232,11359,39743.722449,10987.079643,8615.9,81705.8,25697.2


### 2.4 Permanent Order Table
Dynamic characteristics of payment orders issued to accounts.

| Item | Meaning | Remark |
|:------|:------|:------|
| `order_id` | Unique order identifier ||
| `account_id` | Unique account identifier ||
| `bank_to` | Recipient’s bank | Each bank has a unique two-letter code |
| `account_to` | Recipient’s account ||
| `amount` | Debited amount ||
| `k_symbol` | Payment category | "POJISTNE" = Insurance payment <br> "SIPO" = Household payment <br> "LEASING" = Leasing <br> "UVER" = Loan payment |

In [24]:
order = pd.read_csv('../data/raw-files/order.asc', sep=';', usecols=['account_id', 'amount', 'k_symbol'])

The payment categories align closely with what we already know from the transaction table — insurance and household payments are nothing new. Since every loan account has a permanent order for loan payments, we will cross-check them for leasing payments. If no such accounts exist, this table adds no value to the dataset.

In [25]:
order = order.query('k_symbol in ["LEASING", "UVER"]')
any(order['account_id'].duplicated())

False

### 2.5 Demographic Table
Publicly available informations about the districts.

| Item | Meaning | Remark |
|:------|:------|:------|
| `A1` | District code ||
| `A2` | District name ||
| `A3` | Region ||
| `A4` | Number of inhabitants ||
| ... | ... ||
| `A9` | Number of cities ||
| `A10` | Urban population ratio ||
| `A11` | Average salary ||
| `A12` (`A13`) | Unemployment rate '95 ('96) ||
| `A14` | Entrepreneurs per 1000 inhabitants ||
| `A15` (`A16`) | Number of committed crimes '95 ('96) ||

In [26]:
district = pd.read_csv(
    '../data/raw-files/district.asc', sep=';', usecols=['A1', 'A3', 'A11']
).rename(columns={'A1': 'district_id', 'A3': 'region', 'A11': 'average_salary'})

Each of the 77 districts has a unique mapping across all columns (except _region_), making it tricky to enforce as a logical rule in the ML-based data synthesizer. Plus, _district_ itself is an excessively large categorical feature, which may pose challenges given the relatively small size of our dataset.

A smarter approach could be to group districts by their region and rank them by average salary, reflecting potential economic differences. This ordering also lets us convert _region_ into a numeric feature using ordered label encoding (see the preprocessing section for synthesizers).

In [27]:
district.groupby('region')['average_salary'].mean().sort_values()

region
east Bohemia        8611.181818
south Moravia       8728.500000
south Bohemia       8831.500000
west Bohemia        9015.400000
north Moravia       9049.181818
north Bohemia       9334.200000
central Bohemia     9357.250000
Prague             12541.000000
Name: average_salary, dtype: float64

### 2.6 Credit Card Table
Details of credit cards issued to each account.

| Item | Meaning | Remark |
|:------|:------|:------|
| `card_id` | Unique card identifier ||
| `disp_id` | Unique disposition identifier ||
| `type` | Card type | Possible values: "junior", "classic", "gold" |
| `issued` | Issue date | Format: YYMMDD |

In [28]:
credit_card = pd.read_csv(
    '../data/raw-files/card.asc', sep=';', usecols=['disp_id', 'type']
).rename(columns={'type': 'card'})

### 2.7 Disposition Table
Rights of clients with operating accounts.

| Item | Meaning | Remark |
|:------|:------|:------|
| `disp_id` | Unique disposition identifier ||
| `client_id` | Unique client identifier ||
| `account_id` | Unique account identifier ||
| `type` | Disposition type (owner/user) | Only the owner can issue permanent orders and request a loan |

In [29]:
disp = pd.read_csv(
    '../data/raw-files/disp.asc', sep=';', usecols=['disp_id', 'client_id', 'account_id', 'type']
).rename(columns={'type': 'type_disp'})

In [30]:
# The query fulfills the ownership constraint for loan-related services
disp = disp.query('type_disp == "OWNER"')

### 2.8 Client Table
Client Profile Details.

| Item | Meaning | Remark |
|:------|:------|:------|
| `client_id` | Unique client identifier ||
| `birth_number` | Birthday and Gender | Format: YYMMDD for men <br> Format: YYMM+50DD for women   <br> (YYMMDD represents the date of birth) |
| `district_id` | Client’s residential district ||

In [31]:
client = pd.read_csv('../data/raw-files/client.asc', sep = ';')

In [32]:
def decode_birth_number(dataframe):
    """
    This function decodes the client's birth number to extract gender and date of birth
    
    The birth number consists of six digits (YYMMDD) followed by a four-digit suffix
    - For female clients, 5000 is added to the suffix to indicate gender
    """
    
    # Identifies female clients based on the last four digits
    is_female = dataframe['birth_number'] % 10000 >= 5000
    dataframe['gender'] = np.where(is_female, 'female', 'male')
    
    # Adjusts birth number for females by subtracting 5000
    corrected_birth_numbers = dataframe['birth_number'] - is_female * 5000
    
    # Converts adjusted birth numbers into a proper date of birth
    dataframe['date_of_birth'] = pd.to_datetime(corrected_birth_numbers, format="%y%m%d")
    
    # Fixes incorrect century conversions if necessary
    dataframe.loc[dataframe['date_of_birth'].dt.year >= 2000, 'date_of_birth'] -= pd.DateOffset(years=100)
    
    return dataframe

In [33]:
client = decode_birth_number(client)

## 3.0 Single Table Data Creation

First, we merge our three transaction-related tables into a unified transaction dataset. It first inner joins the _balance_ and _deposit_ tables (both with 234 rows), keeping only matching accounts, then full outer joins the _withdrawal_ table (only 232 rows) to include all accounts, even those with no corresponding withdrawal records.

In [34]:
transaction = (
    balance.merge(deposit, on='account_id')
    .merge(withdrawal, on='account_id', how='outer')
)

Since the full outer join with the _withdrawal_ table introduces missing values for two rows, we need to handle them.

In [35]:
# Fills all NaN values with 0 since these 2 accounts have no withdrawal records  
transaction = transaction.fillna(0)

# Converts the sanction interest column back to its original boolean form (filled 0s become False)  
transaction['sanctions'] = transaction['sanctions'].astype(bool)

In [36]:
owner = (
    disp.merge(credit_card, on ='disp_id', how='outer')
    .merge(client, on ='client_id')
)

loan_data = (
    loan.merge(transaction, on='account_id')
    .merge(owner, on='account_id')
    .merge(account, on='account_id')
    .merge(district, on='district_id')
)

In [37]:
loan_data['age'] = (loan_data['loan_date'] - loan_data['date_of_birth']).dt.days // 365.25

loan_data['relocation'] = loan_data['district_id'] != loan_data['branch_district']

loan_data['account'] = (loan_data['loan_date'] - loan_data['account_date']).dt.days // 7

In [38]:
# Rounding all floating-point columns to two decimal places to maintain precision 
loan_data.update(loan_data.select_dtypes(include='float64').round(2))

In [39]:
# Removes redundant informations that are not necessary for further analysis or modelling
loan_data = loan_data.drop(
    columns=[
        'account_date',
        'birth_number',
        'branch_district',
        'client_id',
        'date_of_birth',
        'disp_id',
        'type_disp',
        'district_id',
        'loan_date',
        'average_salary'
    ]
)

In [40]:
# Exporting our finalized loan dataset in an easy-to-share format without an index
loan_data.to_csv('../data/loan_data.csv', index=False)